In [1]:
### sharp
from utils.utils import *
from utils.data import *
import config
import math

trainer = config.trainer1
model = torch.load(trainer['early_stopping_path'])

In [2]:
images_test = []
for year in config.test_years:
    images_temp = np.memmap(op.join(trainer['path'], f"20d_month_has_vb_[20]_ma_{year}_images.dat"), dtype=np.uint8, mode='r').reshape(
                        (-1, config.IMAGE_HEIGHT[20], config.IMAGE_WIDTH[20]))
    images_test.append(images_temp)
images_test = np.concatenate(images_test)

In [3]:
label_test = []
for year in config.test_years:
    label_temp = pd.read_feather(op.join(trainer['path'], f"20d_month_has_vb_[20]_ma_{year}_labels_w_delay.feather"))
    label_test.append(label_temp)

test_y_all = pd.concat(label_test)
label_test = pd.concat(label_test)[[trainer['target']]]

In [4]:
images_test.shape, test_y_all.shape, label_test.shape

((1502123, 64, 60), (1502123, 35), (1502123, 1))

In [5]:
# train_dataset = CNNDataset(x_train, y_train.values)
# val_dataset = CNNDataset(x_val, y_val.values)
test_dataset = CNNDataset(images_test, label_test.values)

# train_loader = data.DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
# val_loader = data.DataLoader(val_dataset, batch_size=self.batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=trainer['batch_size'], shuffle=False)

In [6]:
# loss_func = torch.nn.CrossEntropyLoss()

In [7]:
prediction = []
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
device_ids=range(torch.cuda.device_count())
global_loss_test = []
with torch.no_grad():
    for images,labels in test_loader:
        test_x = Variable(images.cuda(device=device_ids[3]))
        test_y = Variable(labels.cuda(device=device_ids[3]))
        prediction_test = model(test_x)
        prediction.extend(torch.max(prediction_test,1)[0].cpu().numpy().tolist())
#         loss_test = loss_func(prediction_test, test_y.squeeze().long())
#         global_loss_test.append(loss_test.cpu().detach().numpy())
test_y_all['pred'] = prediction

In [8]:
test_y_all

Date StockID  EWMA_vol      Retx       Retx_5d  Retx_20d  \
0     2000-01-31   10001  0.000659 -0.029851 -3.374006e-07  0.015383   
1     2000-02-29   10001  0.000329  0.007634 -7.576549e-03 -0.045455   
2     2000-03-31   10001  0.000577  0.000000  1.562460e-02  0.031251   
3     2000-04-28   10001  0.000553 -0.018939  1.930500e-02 -0.011582   
4     2000-05-31   10001  0.000344  0.000000 -1.976308e-02 -0.051384   
...          ...     ...       ...       ...           ...       ...   
68632 2019-08-30   93436  0.000867  0.017591  2.739224e-02  0.067639   
68633 2019-09-30   93436  0.000920 -0.005204 -1.307759e-02  0.360527   
68634 2019-10-31   93436  0.002130 -0.000286  6.547814e-02  0.047695   
68635 2019-11-29   93436  0.001075 -0.004075  1.803443e-02  0.256895   
68636 2019-12-31   93436  0.000757  0.008753  1.764403e-01  0.531828   

       Retx_60d  Retx_week  Retx_month  Retx_quarter  Retx_tstat  \
0      0.015383        NaN    0.015383           NaN  -46.020355   
1     -0.022725        NaN   -0.030303           NaN   23.471743   
2     -0.007811   0.015625    0.011720  9.767151e-07    0.000000   
3     -0.011585   0.019305   -0.023165           NaN  -34.100727   
4      0.011857        NaN    0.011857           NaN    0.000000   
...         ...        ...         ...           ...         ...   
68632  0.490805   0.008156    0.067639           NaN   20.296494   
68633  0.765477        NaN    0.307429  7.367476e-01   -5.655738   
68634  0.844879        NaN    0.047695           NaN   -0.134242   
68635  1.057951   0.018034    0.267897           NaN   -3.791517   
68636  0.229557        NaN    0.555159  2.526010e-01   11.563752   

       Retx_5d_tstat  Retx_20d_tstat  Retx_60d_tstat    MarketCap  Retx_label  \
0          -0.000520       23.714886       23.715900     19906.25           0   
1         -23.295103     -139.756775      -69.872169     20212.50           1   
2          26.815258       53.634193      -13.405760    -19712.00           0   
3          34.759727      -20.854383      -20.858652    -19943.00           0   
4         -57.335590     -149.072083       34.398750    -19481.00           0   
...              ...             ...             ...          ...         ...   
68632      31.605164       78.041504      566.290710  40412844.00           1   
68633     -14.212800      391.823242      831.925293  43356600.00           0   
68634      30.733917       22.386717      396.566803  56762756.00           0   
68635      16.779840      239.023972      984.352783  59470036.00           0   
68636     233.098648      702.607605      303.272491  75717728.00           1   

       Retx_5d_label  Retx_20d_label  Retx_60d_label  window_size  \
0                  0               1               1           20   
1                  0               0               0           20   
2                  1               1               0           20   
3                  1               0               0           20   
4                  0               0               1           20   
...              ...             ...             ...          ...   
68632              1               1               1           20   
68633              0               1               1           20   
68634              1               1               1           20   
68635              1               1               1           20   
68636              1               1               1           20   

       next_month_ret_0delay  next_month_ret_1delay       Ret   log_ret  \
0                   0.015383              -0.014927 -0.029851 -0.030306   
1                  -0.030303              -0.030303  0.007634  0.007605   
2                   0.011720               0.031251  0.000000  0.000000   
3                  -0.023165              -0.045454 -0.018939 -0.019121   
4                   0.011857               0.015873  0.000000  0.000000   
...                      ...                    ...       ...       ...   
686

In [17]:
# test_y_all.sort_values(by=['pred'], ascending=False, inplace=True)

In [25]:
# test_y_all['group'] = pd.qcut(test_y_all['pred'], q=10, labels=False)

In [26]:
# test_y_all['group'].value_counts()

group
9    150213
0    150213
2    150213
8    150212
7    150212
6    150212
4    150212
5    150212
3    150212
1    150212
Name: count, dtype: int64

In [30]:
# test_y_all[(test_y_all['group'] == 0)]['Retx_20d'].sum() - test_y_all[(test_y_all['group'] == 9)]['Retx_20d'].sum()

512.0499

In [31]:
# returns = [test_y_all[(test_y_all['group'] == i)]['Retx_20d'].sum() for i in range(0, 9)]

In [35]:
# np.mean(returns), np.std(returns)

(736.78156, 94.96241)

In [41]:
# sharp_ratio = np.mean(returns) / np.std(returns) * math.sqrt(252/20)

In [42]:
# sharp_ratio

27.540528991353654

In [30]:
test = test_y_all

In [31]:
test['group'] = test.groupby('Date')['Retx_20d'].transform(lambda x: pd.qcut(x, 10, labels=False))
test.sort_values(by = 'Date')
date_return = pd.DataFrame(test.groupby(['Date','group'])['Retx_20d'].mean()).reset_index()
group_0_mean = date_return[date_return['group'] == 0]
group_9_mean = date_return[date_return['group'] == 9]
HLreturn = pd.merge(group_0_mean, group_9_mean, on='Date', suffixes=['_group_0', '_group_9'])

HLreturn['diff'] = HLreturn['Retx_20d_group_9'] - HLreturn['Retx_20d_group_0']

mean = HLreturn['diff'].mean()
sd = HLreturn['diff'].std()
sharpe_ratio = mean / sd * math.sqrt(252/20)
sharpe_ratio

11.962855508675398